In [1]:
%run ../chap08/cnn_reg_model.ipynb


In [2]:
class CnnExtModel(CnnRegModel):
    macros = {}
    
    def __init__(self, name, dataset, hconfigs, JJ=0, show_maps=False,
                 l2_decay=0, l1_decay=0, dump_structure=True):
        self.dump_structure = False
        self.layer_index = 0
        self.layer_depth = 0
        self.param_count = 0
        self.inter_stop=0
        self.jongwoo = 1
        self.temp_JJ=JJ
        super(CnnExtModel, self).__init__(name, dataset, hconfigs, show_maps,
                                          l2_decay, l1_decay)
        if self.dump_structure:
            print('Total parameter count: {}'.format(self.param_count))

In [3]:
def cnn_ext_alloc_layer_param(self, input_shape, hconfig):
    layer_type = get_layer_type(hconfig)
    
    
        
    pm, output_shape = super(CnnExtModel, self). \
                          alloc_layer_param(input_shape, hconfig)

    

    return pm, output_shape

CnnExtModel.alloc_layer_param = cnn_ext_alloc_layer_param

In [4]:
def cnn_ext_alloc_parallel_layer(self, input_shape, hconfig):
    pm_hiddens = []
    output_shape = None

    if not isinstance(hconfig[1], dict): hconfig.insert(1, {})
        
    for bconfig in hconfig[2:]:
        bpm, bshape = self.alloc_layer_param(input_shape, bconfig)
        pm_hiddens.append(bpm)
        if output_shape:
            assert output_shape[0:-1] == bshape[0:-1]
            output_shape[-1] += bshape[-1]
        else:
            output_shape = bshape
    
    return {'pms':pm_hiddens}, output_shape
    
def cnn_ext_forward_parallel_layer(self, x, hconfig, pm):
    bys, bauxes, bchns = [], [], []
    for n, bconfig in enumerate(hconfig[2:]):
        by, baux = self.forward_layer(x, bconfig, pm['pms'][n])
        bys.append(by)
        bauxes.append(baux)
        bchns.append(by.shape[-1])
    y = np.concatenate(bys, axis=-1)
    return y, [bauxes, bchns]

def cnn_ext_backprop_parallel_layer(self, G_y, hconfig, pm, aux):
    bauxes, bchns = aux
    bcn_from = 0
    G_x = 0
    for n, bconfig in enumerate(hconfig[2:]):
        bcn_to = bcn_from + bchns[n]
        G_y_slice = G_y[:,:,:,bcn_from:bcn_to]
        G_x += self.backprop_layer(G_y_slice, bconfig, pm['pms'][n], bauxes[n])
    return G_x

CnnExtModel.alloc_parallel_layer = cnn_ext_alloc_parallel_layer
CnnExtModel.forward_parallel_layer = cnn_ext_forward_parallel_layer
CnnExtModel.backprop_parallel_layer = cnn_ext_backprop_parallel_layer

In [5]:
def cnn_ext_alloc_serial_layer(self, input_shape, hconfig):
    pm_hiddens = []
    prev_shape = input_shape

    if not isinstance(hconfig[1], dict): hconfig.insert(1, {})
        
    for sconfig in hconfig[2:]:
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, sconfig)
        pm_hiddens.append(pm_hidden)
    
    return {'pms':pm_hiddens}, prev_shape
    
def cnn_ext_forward_serial_layer(self, x, hconfig, pm):
    hidden = x
    auxes = []

    for n, sconfig in enumerate(hconfig[2:]):
        hidden, aux = self.forward_layer(hidden, sconfig, pm['pms'][n])
        auxes.append(aux)
        
    return hidden, auxes

def cnn_ext_backprop_serial_layer(self, G_y, hconfig, pm, aux):
    auxes = aux
    G_hidden = G_y
    
    for n in reversed(range(len(hconfig[2:]))):
        sconfig, spm, saux = hconfig[2:][n], pm['pms'][n], auxes[n]
        G_hidden = self.backprop_layer(G_hidden, sconfig, spm, saux)
    
    return G_hidden

CnnExtModel.alloc_serial_layer = cnn_ext_alloc_serial_layer
CnnExtModel.forward_serial_layer = cnn_ext_forward_serial_layer
CnnExtModel.backprop_serial_layer = cnn_ext_backprop_serial_layer

In [6]:
def cnn_ext_alloc_add_layer(self, input_shape, hconfig):
    if not isinstance(hconfig[1], dict): hconfig.insert(1, {})
        
    bpm, output_shape = self.alloc_layer_param(input_shape, hconfig[2])
    pm_hiddens = [bpm]

    for bconfig in hconfig[3:]:
        bpm, bshape = self.alloc_layer_param(input_shape, bconfig)
        pm_hiddens.append(bpm)
        check_add_shapes(output_shape, bshape)
    
    if get_conf_param(hconfig, 'x', True):
        check_add_shapes(output_shape, input_shape)
        
    pm = {'pms':pm_hiddens}
    
    for act in get_conf_param(hconfig, 'actions', ''):
        if act == 'B':
            bn_config = ['batch_normal', {'rescale':True}]
            pm['bn'], _ = self.alloc_batch_normal_layer(output_shape,
                        bn_config)

    return pm, output_shape

CnnExtModel.alloc_add_layer = cnn_ext_alloc_add_layer

In [7]:
def cnn_ext_forward_add_layer(self, x, hconfig, pm):
    y, baux = self.forward_layer(x, hconfig[2], pm['pms'][0])
    bauxes, bchns, aux_bn = [baux], [y.shape[-1]], []

    for n, bconfig in enumerate(hconfig[3:]):
        by, baux = self.forward_layer(x, bconfig, pm['pms'][n+1])
        y += tile_add_result(by, y.shape[-1], by.shape[-1])
        bauxes.append(baux)
        bchns.append(by.shape[-1])

    if get_conf_param(hconfig, 'x', True):
        y += tile_add_result(x, y.shape[-1], x.shape[-1])
        
    for act in get_conf_param(hconfig, 'actions', ''):
        if act == 'A': y = self.activate(y, hconfig)
        if act == 'B':
            y, aux_bn = self.forward_batch_normal_layer(y, None, pm['bn'])
            
    return y, [y, bauxes, bchns, aux_bn, x.shape]

CnnExtModel.forward_add_layer = cnn_ext_forward_add_layer

In [8]:
def cnn_ext_backprop_add_layer(self, G_y, hconfig, pm, aux):
    y, bauxes, bchns, aux_bn, x_shape = aux


    for act in reversed(get_conf_param(hconfig, 'actions', '')):
        if act == 'A': G_y = self.activate_derv(G_y, y, hconfig)
        if act == 'B':
            G_y = self.backprop_batch_normal_layer(G_y, None, pm['bn'],
                aux_bn)
    
    G_x = np.zeros(x_shape)
    
    for n, bconfig in enumerate(hconfig[2:]):
        G_by = merge_add_grad(G_y, G_y.shape[-1], bchns[n])
        G_x += self.backprop_layer(G_by, bconfig, pm['pms'][n], bauxes[n])

    if get_conf_param(hconfig, 'x', True):
        G_x += merge_add_grad(G_y, G_y.shape[-1], x_shape[-1])

    return G_x

CnnExtModel.backprop_add_layer = cnn_ext_backprop_add_layer

In [9]:
def check_add_shapes(yshape, bshape):
    assert yshape[:-1] == bshape[:-1]
    assert yshape[-1] % bshape[-1] == 0
    
def tile_add_result(by, ychn, bchn):
    if ychn == bchn: return by
    times = ychn // bchn
    return np.tile(by, times)

def merge_add_grad(G_y, ychn, bchn):
    if ychn == bchn: return G_y
    times = ychn // bchn
    split_shape = G_y.shape[:-1] + tuple([times, bchn])
    return np.sum(G_y.reshape(split_shape), axis=-2)

In [10]:
def cnn_ext_alloc_loop_layer(self, input_shape, hconfig):
    pm_hiddens = []
    prev_shape = input_shape
 
    if not isinstance(hconfig[1], dict): hconfig.insert(1, {})
        
    for n in range(get_conf_param(hconfig, 'repeat', 1)):
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig[2])
        pm_hiddens.append(pm_hidden)
    
    return {'pms':pm_hiddens}, prev_shape
    
def cnn_ext_forward_loop_layer(self, x, hconfig, pm):
    hidden = x
    aux_layers = []

    for n in range(get_conf_param(hconfig, 'repeat', 1)):
        hidden, aux = self.forward_layer(hidden, hconfig[2], pm['pms'][n])
        aux_layers.append(aux)
        
    return hidden, aux_layers

def cnn_ext_backprop_loop_layer(self, G_y, hconfig, pm, aux):
    G_hidden = G_y
    aux_layers = aux
    
    for n in reversed(range(get_conf_param(hconfig, 'repeat', 1))):
        pm_hidden, aux = pm['pms'][n], aux_layers[n]
        G_hidden = self.backprop_layer(G_hidden, hconfig[2], pm_hidden, aux)
    
    return G_hidden

CnnExtModel.alloc_loop_layer = cnn_ext_alloc_loop_layer
CnnExtModel.forward_loop_layer = cnn_ext_forward_loop_layer
CnnExtModel.backprop_loop_layer = cnn_ext_backprop_loop_layer

In [11]:
def cnn_ext_alloc_custom_layer(self, input_shape, hconfig):
    name = get_conf_param(hconfig, 'name')
    args = get_conf_param(hconfig, 'args', {})
    macro = CnnExtModel.get_macro(name, args)

    pm_hidden, output_shape = self.alloc_layer_param(input_shape, macro)
    
    return {'pm':pm_hidden, 'macro':macro}, output_shape
    
def cnn_ext_forward_custom_layer(self, x, hconfig, pm):
    return self.forward_layer(x, pm['macro'], pm['pm'])

def cnn_ext_backprop_custom_layer(self, G_y, hconfig, pm, aux):
    return self.backprop_layer(G_y, pm['macro'], pm['pm'], aux)

CnnExtModel.alloc_custom_layer = cnn_ext_alloc_custom_layer
CnnExtModel.forward_custom_layer = cnn_ext_forward_custom_layer
CnnExtModel.backprop_custom_layer = cnn_ext_backprop_custom_layer

In [ ]:
def cnn_ext_alloc_3d_conv_layer(self, input_shape, hconfig):
    #print(input_shape)
    assert len(input_shape) == 4
    xh, xw, xd, xchn = input_shape
    kh, kw, kd = get_conf_param_3d(hconfig, 'ksize')
    ychn = get_conf_param(hconfig, 'chn')
    
    kernel = np.random.normal(0, self.rand_std, [kh, kw, kd, xchn, ychn], dtype = 'float32')
    #kernel = np.arange(24).reshape((2,2,3,2)).astype(float)
    #print("kernel",kernel[:,:,:,:])
    bias = np.zeros([ychn], dtype = 'float32')

    #print(kernel)
    pm = {'k':kernel, 'b':bias}
    output_shape = [xh, xw, xd, ychn]

    pm['actions'] = get_conf_param(hconfig, 'actions', 'LA')
    for act in pm['actions']:
        if act == 'L':
            input_shape = output_shape
        elif act == 'B':
            bn_config = ['batch_normal', {'rescale':True}]
            #pm['bn'], _ = self.alloc_batch_normal_layer(input_shape, bn_config)

    
    output_shape = eval_stride_shape_3(hconfig, True, xh, xw, xd, ychn)
    return pm, output_shape

CnnExtModel.alloc_3d_conv_layer = cnn_ext_alloc_3d_conv_layer


def cnn_ext_forward_3d_conv_layer(self, x, hconfig, pm):
    y = x
    x_flat, k_flat, relu_y, aux_bn = None, None, None, None
    for act in pm['actions']:
        if act == 'L':
            mb_size, xh, xw, xd, xchn = y.shape
            kh, kw, kd, _, ychn = pm['k'].shape
            
            x_flat = get_3d_ext_regions_for_conv(y, kh, kw, kd)
            k_flat = pm['k'].reshape([kh*kw*kd*xchn, ychn])
           
            conv_flat = np.matmul(x_flat, k_flat)
            y = conv_flat.reshape([mb_size, xh, xw, xd, ychn]) + pm['b']
            #print("!",x_flat.shape)
            #print("!",k_flat.shape)
        elif act == 'A':
            y = self.activate(y, hconfig)
            relu_y = y
        elif act == 'B':
            y, aux_bn = self.forward_batch_normal_layer(y, None, pm['bn'])
    
    y, aux_stride = stride_filter_3(hconfig, True, y)

    if self.need_maps: self.maps.append(y)
    #print(y)
    return y, [x_flat, k_flat, x, relu_y, aux_bn, aux_stride]

CnnExtModel.forward_3d_conv_layer = cnn_ext_forward_3d_conv_layer

def cnn_ext_backprop_3d_conv_layer(self, G_y, hconfig, pm, aux):
    x_flat, k_flat, x, relu_y, aux_bn, aux_stride = aux
    
    G_x = stride_filter_derv_3(hconfig, True, G_y, aux_stride)
    
    for act in reversed(pm['actions']):
        if act == 'L':
            kh, kw, kd, xchn, ychn = pm['k'].shape
            mb_size, xh, xw, xd, _ = G_x.shape

            G_conv_flat = G_x.reshape(mb_size*xh*xw*xd, ychn)
            g_conv_k_flat = x_flat.transpose()
            g_conv_x_flat = k_flat.transpose()
            G_k_flat = np.matmul(g_conv_k_flat, G_conv_flat)
            G_x_flat = np.matmul(G_conv_flat, g_conv_x_flat)
            G_bias = np.sum(G_conv_flat, axis=0)
            G_kernel = G_k_flat.reshape([kh, kw, kd, xchn, ychn])
            G_x = undo_3d_ext_regions_for_conv(G_x_flat, x, kh, kw, kd)
            
            self.update_param(pm, 'k', G_kernel)
            self.update_param(pm, 'b', G_bias)
        elif act == 'A':
            G_x = self.activate_derv(G_x, relu_y, hconfig)
        elif act == 'B':
            G_x = self.backprop_batch_normal_layer(G_x, None, pm['bn'], aux_bn)

    return G_x
    
CnnExtModel.backprop_3d_conv_layer = cnn_ext_backprop_3d_conv_layer

"""
def forward_conv_layer_adhoc(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    for i in range(kh):
                        for j in range(kw):
                            rx = r + i - (kh-1) // 2
                            cx = c + j - (kw-1) // 2
                            if rx < 0 or rx >= xh: continue
                            if cx < 0 or cx >= xw: continue
                            for xm in range(xchn):
                                kval = pm['k'][i][j][xm][ym]
                                ival = x[n][rx][cx][xm]
                                conv[n][r][c][ym] += kval * ival

    y = self.activate(conv + pm['b'], hconfig)
    
    return y, [x, y]
"""

def get_3d_ext_regions_for_conv(x, kh, kw, kd):
    mb_size, xh, xw, xd, xchn = x.shape

    regs = get_3d_ext_regions(x, kh, kw, kd, 0)
    regs = regs.transpose([3, 0, 1, 2, 4, 5, 6, 7])
    #print("3", regs.shape,regs[0])
    return regs.reshape([mb_size*xh*xw*xd, kh*kw*kd*xchn])

def get_3d_ext_regions(x, kh, kw, kd, fill):
    mb_size, xh, xw, xd, xchn = x.shape
    
    eh, ew, ed = xh + kh - 1, xw + kw - 1, xd + kd - 1
    #print(eh,ew,ed)
    bh, bw, bd = (kh-1)//2, (kw-1)//2, (kd-1)//2

    x_ext = np.zeros((mb_size, eh, ew, ed, xchn), dtype = 'float32') + fill
    #print("x_ext",x_ext.shape)
    x_ext[:, bh:bh + xh, bw:bw + xw, bd:bd + xd, :] = x
    #print("4", x_ext.shape,x_ext)
    regs = np.zeros((xh, xw, xd, mb_size*kh*kw*kd*xchn), dtype = 'float32')
    #print("5", regs.shape)
    for r in range(xh):
        for c in range(xw):
            for d_ in range(xd):
                regs[r, c, d_, :] = x_ext[:, r:r + kh, c:c + kw, d_:d_ + kd, :].flatten()
    #print("6", regs.shape,regs[0])
    #print(regs.reshape([xh, xw, xd, mb_size, kh, kw, kd, xchn]).shape)
    return regs.reshape([xh, xw, xd, mb_size, kh, kw, kd, xchn])

def undo_3d_ext_regions_for_conv(regs, x, kh, kw, kd):
    mb_size, xh, xw, xd, xchn = x.shape

    regs = regs.reshape([mb_size, xh, xw, xd, kh, kw, kd, xchn])
    regs = regs.transpose([1, 2, 3, 0, 4, 5, 6, 7])
    
    return undo_3d_ext_regions(regs, kh, kw, kd)

def undo_3d_ext_regions(regs, kh, kw, kd):
    xh, xw, xd, mb_size, kh, kw, kd, xchn = regs.shape
    
    eh, ew, ed = xh + kh - 1, xw + kw - 1, xd + kd - 1
    bh, bw, bd = (kh-1)//2, (kw-1)//2, (kd-1)//2

    gx_ext = np.zeros([mb_size, eh, ew, ed, xchn], dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            for d_ in range(xd):
                gx_ext[:, r:r + kh, c:c + kw, d_:d_ + kd, :] += regs[r, c, d_]

    return gx_ext[:, bh:bh + xh, bw:bw + xw, bd:bd + xd, :]

def get_conf_param_3d(hconfig, key, defval = None):
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    val = hconfig[1][key]
    if isinstance(val, list): return val
    return [val, val, val]



def get_shape_params_3(hconfig, conv_type):
    if conv_type:
        kh, kw, kd = get_conf_param_3d(hconfig, 'ksize')
        sh, sw, sd = get_conf_param_3d(hconfig, 'stride', [1,1,1])
    else:
        sh, sw, sd = get_conf_param_3d(hconfig, 'stride', [1,1,1])
        kh, kw, kd = get_conf_param_3d(hconfig, 'ksize', [sh, sw, sd])
    padding = get_conf_param(hconfig, 'padding', 'SAME')

    return kh, kw, kd, sh, sw, sd, padding

def eval_stride_shape_3(hconfig, conv_type, xh, xw, xd, ychn):
    kh, kw, kd, sh, sw, sd, padding = get_shape_params_3(hconfig, conv_type)
    if padding == 'VALID':
        xh = xh - kh + 1
        xw = xw - kw + 1
        xd = xd - kd + 1
    yh = xh // sh
    yw = xw // sw
    yd = xd // sd
    return [yh, yw, yd, ychn]

def stride_filter_3(hconfig, conv_type, y):
    _, xh, xw, xd, _ = x_shape = y.shape
    nh, nw, nd = xh, xw, xd
    kh, kw, kd, sh, sw, sd, padding = get_shape_params_3(hconfig, conv_type)
    
    if padding == 'VALID':
        bh, bw, bd = (kh-1)//2, (kw-1)//2, (kd-1)//2
        nh, nw, nd = xh - kh + 1, xw - kw + 1, xd - kd + 1
        y = y[:, bh:bh+nh, bw:bw+nw:, bd:bd+nd, :]
    
    if sh != 1 or sw != 1 or sd != 1:
        bh, bw, bd = (sh-1)//2, (sw-1)//2, (sd-1)//2
        mh, mw, md = nh // sh, nw // sw, nd // sd
        y = y[:, bh:bh+mh*sh:sh, bw:bw+mw*sw:sw, bd:bd+md*sd:sd, :]
        
    return y, [x_shape, nh, nw, nd]

def stride_filter_derv_3(hconfig, conv_type, G_y, aux):
    x_shape, nh, nw, nd = aux
    mb_size, xh, xw, xd, chn = x_shape
    kh, kw, kd, sh, sw, sd, padding = get_shape_params_3(hconfig, conv_type)
    
    if sh != 1 or sw != 1 or sd !=1:
        bh, bw, bd = (sh-1)//2, (sw-1)//2, (sd-1)//2
        mh, mw, md = nh // sh, nw // sw, nd // sd
        G_y_tmp = np.zeros([mb_size, nh, nw, nd, chn], dtype = 'float32')
        G_y_tmp[:, bh:bh+mh*sh:sh, bw:bw+mw*sw:sw, bd:bd+md*sd:sd, :] = G_y
        G_y = G_y_tmp
        
    if padding == 'VALID':
        bh, bw, bd = (kh-1)//2, (kw-1)//2, (kd-1)//2
        nh, nw, nd = xh - kh + 1, xw - kw + 1, xd - kd + 1
        G_y_tmp = np.zeros([mb_size, xh, xw, xd, chn], dtype = 'float32')
        G_y_tmp[:, bh:bh+nh, bw:bw+nw:,bd:bd+nd, :] = G_y
        G_y = G_y_tmp
    
    return G_y

def cnn_ext_alloc_3d_max_layer(self, input_shape, hconfig):
    xh, xw, xd, ychn = input_shape
    output_shape = eval_stride_shape_3(hconfig, False, xh, xw, xd, ychn)
    return None, output_shape 

CnnExtModel.alloc_3d_max_layer = cnn_ext_alloc_3d_max_layer

def cnn_ext_forward_3d_max_layer(self, x, hconfig, pm):
    mb_size, xh, xw, xd, chn = x.shape
    sh, sw, sd= get_conf_param_3d(hconfig, 'stride', [1,1])
    kh, kw, kd = get_conf_param_3d(hconfig, 'ksize', [sh, sw, sd])
    #print( sh, sw, kh, kw)
    padding = get_conf_param(hconfig, 'padding', 'SAME')

    if [sh,sw, sd] == [kh, kw, kd] and xh % sh == 0 and xw % sw == 0 and xd % sd == 0 and \
        padding == 'SAME':
        mb_size, xh, xw, xd, chn = x.shape
        sh, sw, sd= get_conf_param_3d(hconfig, 'stride')
        yh, yw, yd= xh // sh, xw // sw, xd // sd

        x1 = x.reshape([mb_size, yh, sh, yw, sw, yd, sd, chn])
        x2 = x1.transpose(0, 1, 3, 5, 7, 2, 4, 6)
        x3 = x2.reshape([-1, sh*sw*sd])

        idxs = np.argmax(x3, axis = 1)
        y_flat = x3[np.arange(mb_size*yh*yw*yd*chn), idxs]
        y = y_flat.reshape([mb_size, yh, yw, yd, chn])

        if self.need_maps: self.maps.append(y)

        return y, idxs

    x_flat = get_3d_ext_regions(x, kh, kw, kd, -np.inf)
    x_flat = x_flat.transpose([3,7,0,1,2,4,5,6])
    x_flat = x_flat.reshape(mb_size*chn*xh*xw*xd,kh*kw*kd)

    max_idx = np.argmax(x_flat, axis=1)
    y = x_flat[np.arange(x_flat.shape[0]), max_idx]
    y = y.reshape([mb_size, chn, xh, xw,xd])
    y = y.transpose([0,2,3,4,1])

    y, aux_stride = stride_filter_3(hconfig, False, y)
    
    if self.need_maps: self.maps.append(y)
        
    return y, [x.shape, kh, kw, kd, sh, sw, sd, padding, max_idx, aux_stride]

CnnExtModel.forward_3d_max_layer = cnn_ext_forward_3d_max_layer

def cnn_ext_backprop_3d_max_layer(self, G_y, hconfig, pm, aux):
    if not isinstance(aux, list):
        idxs = aux
    
        mb_size, yh, yw,yd, chn = G_y.shape
        sh, sw,sd = get_conf_param_3d(hconfig, 'stride')
        xh, xw, xd = yh * sh, yw * sw, yd * sd

        gy_flat = G_y.flatten()

        gx1 = np.zeros([mb_size*yh*yw*yd*chn, sh*sw*sd], dtype = 'float32')
        gx1[np.arange(mb_size*yh*yw*yd*chn), idxs] = gy_flat[:]
        gx2 = gx1.reshape([mb_size, yh, yw, yd, chn, sh, sw, sd])
        gx3 = gx2.transpose([0, 1, 5, 2, 6, 3, 7, 4])

        G_input = gx3.reshape([mb_size, xh, xw, xd, chn])

        return G_input
    
    x_shape, kh, kw, kd, sh, sw, sd, padding, max_idx, aux_stride = aux
    mb_size, xh, xw, xd, chn = x_shape
    
    G_y = stride_filter_derv_3(hconfig, False, G_y, aux_stride)
    
    G_y = G_y.transpose([0,4,1,2,3])
    G_y = G_y.flatten()
    
    G_x_flat = np.zeros([mb_size*chn*xh*xw*xd, kh*kw*kd], dtype = 'float32')
    G_x_flat[np.arange(G_x_flat.shape[0]), max_idx] = G_y

    G_x_flat = G_x_flat.reshape(mb_size, chn, xh, xw, xd, kh, kw, kd)
    G_x_flat = G_x_flat.transpose([2,3,4,0,5,6,7,1])
    G_x = undo_3d_ext_regions(G_x_flat, kh, kw, kd)

    return G_x

CnnExtModel.backprop_3d_max_layer = cnn_ext_backprop_3d_max_layer

In [12]:
def cnn_ext_set_macro(name, config):
    CnnExtModel.macros[name] = config
    
def cnn_ext_get_macro(name, args):
    restored = copy.deepcopy(CnnExtModel.macros[name])
    replace_arg(restored, args)
        
    return restored
    
def replace_arg(exp, args):
    if isinstance(exp, (list, tuple)):
        for n, term in enumerate(exp):
            if isinstance(term, str) and term[0] == '#':
                if term[1] == '#': exp[n] = term[1:]
                elif term in args: exp[n] = args[term]
            else:
                replace_arg(term, args)
    elif isinstance(exp, dict):
        for key in exp:
            if isinstance(exp[key], str) and exp[key][0] == '#':
                if exp[key][1] == '#': exp[key] = exp[key][1:]
                elif exp[key] in args: exp[key] = args[exp[key]]
            else:
                replace_arg(exp[key], args)
                
CnnExtModel.set_macro = cnn_ext_set_macro
CnnExtModel.get_macro = cnn_ext_get_macro

In [ ]:
def cnn_ext_set_macro(name, config):
    CnnExtModel.macros[name] = config
    
def cnn_ext_get_macro(name, args):
    restored = copy.deepcopy(CnnExtModel.macros[name])
    replace_arg(restored, args)
        
    return restored
    
def replace_arg(exp, args):
    if isinstance(exp, (list, tuple)):
        for n, term in enumerate(exp):
            if isinstance(term, str) and term[0] == '#':
                if term[1] == '#': exp[n] = term[1:]
                elif term in args: exp[n] = args[term]
            else:
                replace_arg(term, args)
    elif isinstance(exp, dict):
        for key in exp:
            if isinstance(exp[key], str) and exp[key][0] == '#':
                if exp[key][1] == '#': exp[key] = exp[key][1:]
                elif exp[key] in args: exp[key] = args[exp[key]]
            else:
                replace_arg(exp[key], args)
                
CnnExtModel.set_macro = cnn_ext_set_macro
CnnExtModel.get_macro = cnn_ext_get_macro

In [13]:
def cnn_ext_alloc_conv_layer(self, input_shape, hconfig):
    pm, output_shape = super(CnnExtModel, self). \
                           alloc_conv_layer(input_shape, hconfig)
    pm['actions'] = get_conf_param(hconfig, 'actions', 'LAB')
    
    self.layer_type="conv"
    for act in pm['actions']:
        if act == 'L':
            input_shape = output_shape
        elif act == 'B':
            bn_config = ['batch_normal', {'rescale':True}]
            pm['bn'], _ = self.alloc_batch_normal_layer(input_shape, bn_config)
    xh, xw, xchn = input_shape
    ychn = get_conf_param(hconfig, 'chn')
    output_shape = eval_stride_shape(hconfig, True, xh, xw, ychn)
    
    return pm, output_shape

CnnExtModel.alloc_conv_layer = cnn_ext_alloc_conv_layer

In [14]:
def cnn_ext_forward_conv_layer(self, x, hconfig, pm):
    y = x
    x_flat, k_flat, relu_y, aux_bn = None, None, None, None
    for act in pm['actions']:
        if act == 'L':
            mb_size, xh, xw, xchn = y.shape
            kh, kw, _, ychn = pm['k'].shape
            x_flat = get_ext_regions_for_conv(y, kh, kw)
            k_flat = pm['k'].reshape([kh*kw*xchn, ychn])
            conv_flat = np.matmul(x_flat, k_flat)
            y = conv_flat.reshape([mb_size, xh, xw, ychn]) + pm['b']
        elif act == 'A':
            y = self.activate(y, hconfig)
            relu_y = y
        elif act == 'B':
            y, aux_bn = self.forward_batch_normal_layer(y, None, pm['bn'])

    y, aux_stride = stride_filter(hconfig, True, y)

    if self.need_maps: self.maps.append(y)
        
    return y, [x_flat, k_flat, x, relu_y, aux_bn, aux_stride]

CnnExtModel.forward_conv_layer = cnn_ext_forward_conv_layer

In [15]:
def cnn_ext_backprop_conv_layer(self, G_y, hconfig, pm, aux):
    x_flat, k_flat, x, relu_y, aux_bn, aux_stride = aux
    
    G_x = stride_filter_derv(hconfig, True, G_y, aux_stride)
    
    for act in reversed(pm['actions']):
        if act == 'L':
            kh, kw, xchn, ychn = pm['k'].shape
            mb_size, xh, xw, _ = G_x.shape

            G_conv_flat = G_x.reshape(mb_size*xh*xw, ychn)
            g_conv_k_flat = x_flat.transpose()
            g_conv_x_flat = k_flat.transpose()
            G_k_flat = np.matmul(g_conv_k_flat, G_conv_flat)
            G_x_flat = np.matmul(G_conv_flat, g_conv_x_flat)
            G_bias = np.sum(G_conv_flat, axis=0)
            G_kernel = G_k_flat.reshape([kh, kw, xchn, ychn])
            G_x = undo_ext_regions_for_conv(G_x_flat, x, kh, kw)
            
            self.update_param(pm, 'k', G_kernel)
            self.update_param(pm, 'b', G_bias)
        elif act == 'A':
            G_x = self.activate_derv(G_x, relu_y, hconfig)
        elif act == 'B':
            G_x = self.backprop_batch_normal_layer(G_x, None, pm['bn'], aux_bn)

    return G_x
    
CnnExtModel.backprop_conv_layer = cnn_ext_backprop_conv_layer

In [17]:
def cnn_ext_alloc_max_layer(self, input_shape, hconfig):
    xh, xw, ychn = input_shape
    output_shape = eval_stride_shape(hconfig, False, xh, xw, ychn)
    return None, output_shape 

CnnExtModel.alloc_max_layer = cnn_ext_alloc_max_layer

In [18]:
def cnn_ext_forward_max_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride', [1,1])
    kh, kw = get_conf_param_2d(hconfig, 'ksize', [sh, sw])
    #print( sh, sw, kh, kw)
    padding = get_conf_param(hconfig, 'padding', 'SAME')

    if [sh,sw] == [kh, kw] and xh % sh == 0 and xw % sw == 0 and \
        padding == 'SAME':
        return super(CnnExtModel, self).forward_max_layer(x, hconfig, pm)

    x_flat = get_ext_regions(x, kh, kw, -np.inf)
    x_flat = x_flat.transpose([2,5,0,1,3,4])
    x_flat = x_flat.reshape(mb_size*chn*xh*xw,kh*kw)

    max_idx = np.argmax(x_flat, axis=1)
    y = x_flat[np.arange(x_flat.shape[0]), max_idx]
    y = y.reshape([mb_size, chn, xh, xw])
    y = y.transpose([0,2,3,1])

    y, aux_stride = stride_filter(hconfig, False, y)
    
    if self.need_maps: self.maps.append(y)
        
    return y, [x.shape, kh, kw, sh, sw, padding, max_idx, aux_stride]

CnnExtModel.forward_max_layer = cnn_ext_forward_max_layer

In [19]:
def cnn_ext_backprop_max_layer(self, G_y, hconfig, pm, aux):
    if not isinstance(aux, list):
        return super(CnnExtModel, self).backprop_max_layer(G_y, hconfig,
                pm, aux)
    
    x_shape, kh, kw, sh, sw, padding, max_idx, aux_stride = aux
    mb_size, xh, xw, chn = x_shape
    
    G_y = stride_filter_derv(hconfig, False, G_y, aux_stride)
    
    G_y = G_y.transpose([0,3,1,2])
    G_y = G_y.flatten()
    
    G_x_flat = np.zeros([mb_size*chn*xh*xw, kh*kw], dtype = 'float32')
    G_x_flat[np.arange(G_x_flat.shape[0]), max_idx] = G_y

    G_x_flat = G_x_flat.reshape(mb_size, chn, xh, xw, kh, kw)
    G_x_flat = G_x_flat.transpose([2,3,0,4,5,1])
    G_x = undo_ext_regions(G_x_flat, kh, kw)

    return G_x

CnnExtModel.backprop_max_layer = cnn_ext_backprop_max_layer

In [20]:
def cnn_ext_alloc_avg_layer(self, input_shape, hconfig):
    xh, xw, chn = input_shape
    sh, sw = get_conf_param_2d(hconfig, 'stride', [1,1])
    kh, kw = get_conf_param_2d(hconfig, 'ksize', [sh, sw])
    padding = get_conf_param(hconfig, 'padding', 'SAME')

    if [sh,sw] == [kh, kw] and xh % sh == 0 and xw % sw == 0 \
                   and padding == 'SAME':
        return super(CnnExtModel, self).alloc_avg_layer(input_shape, hconfig)

    one_mask = np.ones([1, xh, xw, chn], dtype = 'float32')
    
    m_flat = get_ext_regions(one_mask, kh, kw, 0)
    m_flat = m_flat.transpose([2,5,0,1,3,4])
    m_flat = m_flat.reshape(1*chn*xh*xw,kh*kw)

    mask = np.sum(m_flat, axis=1)
    
    output_shape = eval_stride_shape(hconfig, False, xh, xw, chn)

    return {'mask':mask}, output_shape 

CnnExtModel.alloc_avg_layer = cnn_ext_alloc_avg_layer

In [21]:
def cnn_ext_forward_avg_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride', [1,1])
    kh, kw = get_conf_param_2d(hconfig, 'ksize', [sh, sw])
    padding = get_conf_param(hconfig, 'padding', 'SAME')

    if [sh,sw] == [kh, kw] and xh % sh == 0 and xw % sw == 0 \
                   and padding == 'SAME':
        return super(CnnExtModel, self).forward_avg_layer(x, hconfig, pm)

    x_flat = get_ext_regions(x, kh, kw, 0)
    x_flat = x_flat.transpose([2,5,0,1,3,4])
    x_flat = x_flat.reshape(mb_size*chn*xh*xw,kh*kw)

    hap = np.sum(x_flat, axis=1)
    
    y = np.reshape(hap, [mb_size, -1]) / pm['mask']
    y = y.reshape([mb_size, chn, xh, xw])
    y = y.transpose([0,2,3,1])

    y, aux_stride = stride_filter(hconfig, False, y)
    
    if self.need_maps: self.maps.append(y)
        
    return y, [x.shape, kh, kw, sh, sw, padding, aux_stride]

CnnExtModel.forward_avg_layer = cnn_ext_forward_avg_layer

In [22]:
def cnn_ext_backprop_avg_layer(self, G_y, hconfig, pm, aux):
    if not isinstance(aux, list):
        return super(CnnExtModel, self).backprop_avg_layer(G_y, hconfig, pm, aux)
    
    x_shape, kh, kw, sh, sw, padding, aux_stride = aux
    mb_size, xh, xw, chn = x_shape
    
    G_y = stride_filter_derv(hconfig, False, G_y, aux_stride)
    
    G_y = G_y.transpose([0,3,1,2])
    G_y = G_y.flatten()
    
    G_hap = np.reshape(G_y, [mb_size, -1]) / pm['mask']
    G_x_flat = np.tile(G_hap, (kh*kw, 1))

    G_x_flat = G_x_flat.reshape(mb_size, chn, xh, xw, kh, kw)
    G_x_flat = G_x_flat.transpose([2,3,0,4,5,1])
    G_x = undo_ext_regions(G_x_flat, kh, kw)

    return G_x

CnnExtModel.backprop_avg_layer = cnn_ext_backprop_avg_layer

In [23]:
def eval_stride_shape(hconfig, conv_type, xh, xw, ychn):
    kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)
    if padding == 'VALID':
        xh = xh - kh + 1
        xw = xw - kw + 1
    yh = xh // sh
    yw = xw // sw
    return [yh, yw, ychn]

In [24]:
def stride_filter(hconfig, conv_type, y):
    _, xh, xw, _ = x_shape = y.shape
    nh, nw = xh, xw
    kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)
    
    if padding == 'VALID':
        bh, bw = (kh-1)//2, (kw-1)//2
        nh, nw = xh - kh + 1, xw - kw + 1
        y = y[:, bh:bh+nh, bw:bw+nw:, :]
    
    if sh != 1 or sw != 1:
        bh, bw = (sh-1)//2, (sw-1)//2
        mh, mw = nh // sh, nw // sw
        y = y[:, bh:bh+mh*sh:sh, bw:bw+mw*sw:sw, :]
        
    return y, [x_shape, nh, nw]

In [25]:
def stride_filter_derv(hconfig, conv_type, G_y, aux):
    x_shape, nh, nw = aux
    mb_size, xh, xw, chn = x_shape
    kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)
    
    if sh != 1 or sw != 1:
        bh, bw = (sh-1)//2, (sw-1)//2
        mh, mw = nh // sh, nw // sw
        G_y_tmp = np.zeros([mb_size, nh, nw, chn], dtype = 'float32')
        G_y_tmp[:, bh:bh+mh*sh:sh, bw:bw+mw*sw:sw, :] = G_y
        G_y = G_y_tmp
        
    if padding == 'VALID':
        bh, bw = (kh-1)//2, (kw-1)//2
        nh, nw = xh - kh + 1, xw - kw + 1
        G_y_tmp = np.zeros([mb_size, xh, xw, chn], dtype = 'float32')
        G_y_tmp[:, bh:bh+nh, bw:bw+nw:, :] = G_y
        G_y = G_y_tmp
    
    return G_y

In [26]:
def get_shape_params(hconfig, conv_type):
    if conv_type:
        kh, kw = get_conf_param_2d(hconfig, 'ksize')
        sh, sw = get_conf_param_2d(hconfig, 'stride', [1,1])
    else:
        sh, sw = get_conf_param_2d(hconfig, 'stride', [1,1])
        kh, kw = get_conf_param_2d(hconfig, 'ksize', [sh, sw])
    padding = get_conf_param(hconfig, 'padding', 'SAME')

    return kh, kw, sh, sw, padding